# PART-1: BASIC CLASS STRUCTURE


In [1]:
class Task:
    """ A class to represent a single task in a project. """
    # 1. Initialize with task_id, title, description, and status
    def __init__(self, task_id: int, title: str, description: str = "", status: str = "pending"):
        self.task_id = task_id          
        self.title = title                
        self.description = description    
        self.status = status              

    # 2. Change status (but only if valid)
    def update_status(self, new_status: str):
        valid_statuses = ["pending", "in_progress", "completed"]
        if new_status in valid_statuses:
            self.status = new_status
        else:
            print("Invalid status.")

    # 3. Convert to a string like: "1|Write Report|About AI|completed"
    def to_string(self) -> str:
        return f"{self.task_id}|{self.title}|{self.description}|{self.status}"

    # 4. Turn string back into Task object
    @classmethod
    def from_string(cls, data_line: str):
        parts = data_line.strip().split('|')
        if len(parts) != 4:
            raise ValueError("Error: Line must have 4 parts separated by |")
        task_id = int(parts[0])  # Convert string "1" to int 1
        title = parts[1]
        description = parts[2]
        status = parts[3]
        return cls(task_id, title, description, status)

    # 5. Show task details
    def __str__(self):
        return f"ID {self.task_id} |Title : {self.title} |Description : {self.description} |Status : {self.status}"

## Testing the method

In [2]:
task1 = Task(1, "Design UI", "Design the homepage")
print(task1)  # Shows the task

task1.update_status("in_progress")  # Changes status
print(task1.to_string())            # For saving to file

task_line = "2|Write Backend|Create APIs|pending"
task2 = Task.from_string(task_line)
print(task2)

ID 1 |Title : Design UI |Description : Design the homepage |Status : pending
1|Design UI|Design the homepage|in_progress
ID 2 |Title : Write Backend |Description : Create APIs |Status : pending


## PART 2: PROJECT CLASS WITH FILE HANDLING 

In [3]:
from typing import List, Optional

class Project:
    """ A class to represent a software project containing multiple tasks. """

    # 1. Initialize the project
    def __init__(self, name: str, description: str = ""):
        self.name = name
        self.description = description
        self.tasks: List[Task] = []       # to store task objects
        self.next_task_id = 1             # start task IDs from 1

    # 2. Add a new task
    def add_task(self, title: str, description: str = "") -> bool:
        try:
            task = Task(self.next_task_id, title, description)
            self.tasks.append(task)
            self.next_task_id += 1
            return True
        except Exception as e:
            print(f"Failed to add task: {e}")
            return False

    # 3. Get task by ID
    def get_task(self, task_id: int) -> Optional[Task]:
        for task in self.tasks:
            if task.task_id == task_id:
                return task
        return None

    # 4. Update status of a task
    def update_task_status(self, task_id: int, new_status: str) -> bool:
        try:
            task = self.get_task(task_id)
            if task is not None:
                task.update_status(new_status)
                return True
            else:
                print(f"Task with ID {task_id} not found")
                return False
        except Exception as e:
            print(f"Failed to update status: {e}")
            return False

    # 5️⃣ List all tasks
    def list_tasks(self) -> List[Task]:
        return self.tasks

    # 6️⃣ Filter tasks by status
    def get_tasks_by_status(self, status: str) -> List[Task]:
        return [task for task in self.tasks if task.status == status]


## Testing the method

In [4]:
project = Project("My Web App", "Build a cool app")

project.add_task("Setup Database", "Create schema")
project.add_task("Build UI", "Frontend layout")
project.update_task_status(1, "in_progress")

# Show all tasks
for task in project.list_tasks():
    print(task)

# Show only pending tasks
print("\n Pending Tasks:")
for task in project.get_tasks_by_status("pending"):
    print(task)

ID 1 |Title : Setup Database |Description : Create schema |Status : in_progress
ID 2 |Title : Build UI |Description : Frontend layout |Status : pending

 Pending Tasks:
ID 2 |Title : Build UI |Description : Frontend layout |Status : pending


## PART 3: FILE HANDLING WITH EXCEPTION HANDLING 

In [6]:
import os
from typing import List, Optional

class ProjectManager:
    """ Manages multiple projects and handles file saving/loading """

    # 1. Constructor: Initialize with folder name and create folder
    def __init__(self, data_directory: str = "projects"):
        self.data_directory = data_directory
        self.create_directory()

    # 2️⃣ Create directory if not exists
    def create_directory(self):
        try:
            os.makedirs(self.data_directory, exist_ok=True)
        except PermissionError:
            print("Permission denied: Cannot create the directory.")
        except Exception as e:
            print(f"Error creating directory: {e}")

    # 3️⃣ Save a project to file
    def save_project(self, project: Project) -> bool:
        try:
            filename = f"{project.name.replace(' ', '_')}.txt"
            filepath = os.path.join(self.data_directory, filename)

            with open(filepath, 'w') as file:
                file.write(f"PROJECT_INFO|{project.name}|{project.description}\n")
                for task in project.tasks:
                    file.write(f"TASK|{task.to_string()}\n")

            print(f"✅ Project '{project.name}' saved to {filepath}")
            return True

        except FileNotFoundError:
            print("❌ File path not found.")
            return False
        except PermissionError:
            print("❌ No permission to write file.")
            return False
        except Exception as e:
            print(f"❌ Error saving project: {e}")
            return False

    # 4️⃣ Load a project from file
    def load_project(self, project_name: str) -> Optional[Project]:
        try:
            filename = f"{project_name.replace(' ', '_')}.txt"
            filepath = os.path.join(self.data_directory, filename)

            with open(filepath, 'r') as file:
                lines = file.readlines()

            project = None
            for line in lines:
                parts = line.strip().split('|')
                if parts[0] == "PROJECT_INFO":
                    _, name, description = parts
                    project = Project(name, description)
                elif parts[0] == "TASK" and project is not None:
                    task_data = '|'.join(parts[1:])  # get task string
                    task = Task.from_string(task_data)
                    project.tasks.append(task)
                    project.next_task_id += 1

            print(f"✅ Project '{project_name}' loaded successfully")
            return project

        except FileNotFoundError:
            print("❌ File not found.")
            return None
        except ValueError:
            print("❌ Data in file is malformed.")
            return None
        except Exception as e:
            print(f"❌ Error loading project: {e}")
            return None

    # 5️⃣ List all saved project files
    def list_saved_projects(self) -> List[str]:
        try:
            files = os.listdir(self.data_directory)
            projects = [f[:-4] for f in files if f.endswith('.txt')]
            return projects

        except FileNotFoundError:
            print("❌ Project directory not found.")
            return []
        except Exception as e:
            print(f"❌ Error listing projects: {e}")
            return []

    # 6️⃣ Delete a project file
    def delete_project(self, project_name: str) -> bool:
        try:
            filename = f"{project_name.replace(' ', '_')}.txt"
            filepath = os.path.join(self.data_directory, filename)
            os.remove(filepath)
            print(f"✅ Project '{project_name}' deleted successfully.")
            return True

        except FileNotFoundError:
            print(" Project file not found.")
            return False
        except PermissionError:
            print(" No permission to delete this file.")
            return False
        except Exception as e:
            print(f" Error deleting project: {e}")
            return False


## PART-4: Demonstration and Testing

In [9]:
def demonstrate_system():
    """ Demonstrate the project management system. """
    print("=== PROJECT MANAGEMENT SYSTEM DEMO ===\n")

    # 1️⃣ Create ProjectManager instance
    pm = ProjectManager()

    # 2️⃣ Create a new Project
    project = Project("My Web App", "A cool web application")

    # 3️⃣ Add some tasks
    project.add_task("Setup database", "Create database schema")
    project.add_task("Design UI", "Design homepage and login page")
    project.add_task("Build Backend", "Set up server and APIs")

    # 4️⃣ Display all tasks
    print("\n Current Tasks:")
    for task in project.list_tasks():
        print(task)

    # 5️⃣ Update status of a task
    project.update_task_status(1, "in_progress")
    project.update_task_status(2, "completed")

    # 6️⃣ Save the project to a file
    pm.save_project(project)

    # 7️⃣ Load the project again from file
    loaded_project = pm.load_project("My Web App")

    # 8️⃣ Display loaded project tasks
    if loaded_project:
        print("\n Loaded Project Info:")
        print(f"Name: {loaded_project.name}")
        print(f"Description: {loaded_project.description}")
        print("\n Tasks in loaded project:")
        for task in loaded_project.list_tasks():
            print(task)

    print("\n=== DEMO COMPLETED ===")


In [13]:
if __name__ == "__main__":
    demonstrate_system()
    # Custom test
    my_project = Project("Extra Project", "Testing more features")
    my_project.add_task("Test feature", "Ensure system works")
    for task in my_project.list_tasks():
        print(task)


=== PROJECT MANAGEMENT SYSTEM DEMO ===


 Current Tasks:
ID 1 |Title : Setup database |Description : Create database schema |Status : pending
ID 2 |Title : Design UI |Description : Design homepage and login page |Status : pending
ID 3 |Title : Build Backend |Description : Set up server and APIs |Status : pending
✅ Project 'My Web App' saved to projects\My_Web_App.txt
✅ Project 'My Web App' loaded successfully

 Loaded Project Info:
Name: My Web App
Description: A cool web application

 Tasks in loaded project:
ID 1 |Title : Setup database |Description : Create database schema |Status : in_progress
ID 2 |Title : Design UI |Description : Design homepage and login page |Status : completed
ID 3 |Title : Build Backend |Description : Set up server and APIs |Status : pending

=== DEMO COMPLETED ===
ID 1 |Title : Test feature |Description : Ensure system works |Status : pending
